In [105]:
import fitz  # PyMuPDF

def extract_lines_from_pdf_pymupdf(pdf_file):
    """
    Extrae las líneas completas de texto de un archivo PDF utilizando PyMuPDF.
    
    Args:
        pdf_path (str): Ruta del archivo PDF.
        
    Returns:
        list: Lista de líneas extraídas del PDF.
    """
    lines = []
    with fitz.open(pdf_file) as doc:
        for page in doc:
            text = page.get_text("text")  # Obtener texto como líneas completas
            page_lines = text.splitlines()
            lines.extend([line.strip() for line in page_lines if line.strip()])
    return lines

pdf_path = "./Dialnet-LaViolenciaComoOpcionCreativaEnLaObraDeMartinScors-7009818.pdf"  
pdf_lines = extract_lines_from_pdf_pymupdf(pdf_path)
print(pdf_lines)

['AISTHESIS N° 37, 2004. pp 64-85', 'ESTUDIOS', 'LA VIOLENCIA COMO OPCIÓN CREATIVA', 'EN LA OBRA DE MARTÍN SCORSESE', 'Luis E. Cécéreu Lagos', 'Instituto de Estética', 'Facultad de Filosofía', 'Pontificia Universidad Católica de Chile', 'Resumen', 'Martín Scorsese es uno de los autores más prestigiosos de la cinematografía actual. La', 'densidad conceptual de sus filmes. Su estilística, definida en una rigurosa y personal', 'factura. La diversidad de relaciones entre la tradición cinematográfica, como referentes', 'en sus obras, constituyen algunos de los aportes que exhibe a través de una extensa', 'filmografía.', 'La poética de la violencia. La consideración de ella como forma de vida. Las vivencias', 'personales, referidas como códigos privados de información. Sus inquietudes religiosas.', 'La obra cinematográfica como reflexión de su propio lenguaje.', 'Dichos factores son descritos en el presente trabajo, como elementos inherentes a la', 'obra de Scorsese, los que tendrán en el fi

In [106]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [107]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=pdf_lines,
    collection_name="pdf-prueba6",
    embedding=embeddings,
)

In [108]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdf-prueba6",
    embedding_function=embeddings,
)

In [109]:
retriever = vector_store.as_retriever()

In [110]:
question = "Como me llamo?"
docs = vector_store.similarity_search(question)
len(docs)

4

In [111]:
docs[0]

Document(metadata={}, page_content='¿Lo dices por mí u os dijeron algo?')

In [112]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [113]:
def search_chroma(input,history):
    try:
        # Invocar la cadena con la consulta como input
        response = chain.invoke(input)
        return response  
    except Exception as e:
        # Manejo de errores
        return f"Error al procesar la consulta: {e}"

Tip:
 Always set type="messages" in gr.ChatInterface. The default value (type="tuples") is deprecated and will be removed in a future version of Gradio.

In [114]:
# Definir la interfaz de chat
import gradio as gr
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown("### Chat con llama3.2")
    
    # Crear el ChatInterface para manejar la interacción
    chat_interface = gr.ChatInterface(fn=search_chroma,type="messages",
                                       examples=["Quien es Martin Scorsese", "Que relación tiene la violencia es sus películas?", "Háblame de Goodfellas"],
                                       )

# Lanzar la interfaz
demo.launch()

* Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.
